In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import concatenate, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.set_random_seed(seed)

In [2]:
train = np.load('data/train.npy', allow_pickle = 'True')
test = np.load('data/test.npy', allow_pickle = 'True')

In [3]:
x = train[:,2:]
x = np.reshape(x, (-1, 28, 28, 1))

x_letter = train[:,1]
x_letter = np.reshape(x_letter, (-1, 1))
en = OneHotEncoder()
x_letter = en.fit_transform(x_letter).toarray()

y = train[:,0]
y = np.reshape(y, (-1, 1))
en = OneHotEncoder()
y = en.fit_transform(y).toarray()

print(x.shape)
print(x_letter.shape)
print(y.shape)

(2048, 28, 28, 1)
(2048, 26)
(2048, 10)


In [4]:
valid_size = 48
valid_x = x[-valid_size:]
x = x[:-48]

valid_x_letter = x_letter[-valid_size:]
x_letter = x_letter[:-48]

valid_y = y[-valid_size:]
y = y[:-48]

print(x.shape)
print(valid_x.shape)
print(x_letter.shape)
print(valid_x_letter.shape)
print(y.shape)
print(valid_y.shape)

(2000, 28, 28, 1)
(48, 28, 28, 1)
(2000, 26)
(48, 26)
(2000, 10)
(48, 10)


In [5]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     shear_range=10)

In [6]:
x_total = x.copy()
def augment(x):
    aug_list = []
    for i in range(x.shape[0]):
        num_aug = 0
        tmp = x[i]
        tmp = tmp.reshape((1,) + tmp.shape)
        for x_aug in image_generator.flow(tmp, batch_size = 1) :
            if num_aug >= 1:
                break
            aug_list.append(x_aug[0])
            num_aug += 1
    aug_list = np.array(aug_list)
    return aug_list

n = 2
for i in range(n):
    arr = augment(x)
    x_total = np.concatenate((x_total, arr), axis=0)
    if i > n:
        break

print(x_total.shape)

(6000, 28, 28, 1)


In [7]:
y_total = y.copy()
for i in range(n):
    arr = y.copy()
    y_total = np.concatenate((y_total, arr), axis=0)

print(y_total.shape)

(6000, 10)


In [8]:
x_letter_total = x_letter.copy()
for i in range(n):
    arr = x_letter.copy()
    x_letter_total = np.concatenate((x_letter_total, arr), axis=0)
    
print(x_letter_total.shape)

(6000, 26)


In [9]:
def Conv_block(x, growth_rate, activation='relu'):
    x_l = BatchNormalization()(x)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate*4, (1,1), padding='same', kernel_initializer='he_normal')(x_l)
    
    x_l = BatchNormalization()(x_l)
    x_l = Activation(activation)(x_l)
    x_l = Conv2D(growth_rate, (3,3), padding='same', kernel_initializer='he_normal')(x_l)
    
    x = concatenate([x, x_l])
    return x

def Dense_block(x, layers, growth_rate=32):
    for i in range(layers):
        x = Conv_block(x, growth_rate)
    return x

def Transition_layer(x, compression_factor=0.5, activation='relu'):
    reduced_filters = int(tf.keras.backend.int_shape(x)[-1] * compression_factor)
    
    x = BatchNormalization()(x)
    x = Activation(activation)(x)
    x = Conv2D(reduced_filters, (1,1), padding='same', kernel_initializer='he_normal')(x)
    
    x = AveragePooling2D((2,2), padding='same', strides=2)(x)
    return x

def DenseNet(densenet_type='DenseNet-121', base_growth_rate=32):
    model_input = Input(shape=(28,28,1))
    x = Conv2D(base_growth_rate*2, (5,5), padding='same', strides=1,
               kernel_initializer='he_normal')(model_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2,2), padding='same', strides=1)(x)
    
    x = Dense_block(x, layers_in_block[densenet_type][0], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][1], base_growth_rate)
    x = Transition_layer(x, compression_factor=0.5)
    x = Dense_block(x, layers_in_block[densenet_type][2], base_growth_rate)
    #x = Transition_layer(x, compression_factor=0.5)
    #x = Dense_block(x, layers_in_block[densenet_type][3], base_growth_rate)
    
    x = GlobalAveragePooling2D()(x)
    
    model_output = Dense(10, activation='softmax', kernel_initializer='he_normal')(x)
    
    model = Model(model_input, model_output, name=densenet_type)
    model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy')
    
    return model

In [10]:
def hyperparameters():
    base_growth_rate = [8, 16, 32]
    batches = [32, 64]
    return {'base_growth_rate':base_growth_rate, 'batch_size':batches}

hyperparameters = hyperparameters()

In [11]:
layers_in_block = {'DenseNet-121':[6, 12, 24, 16],
                   'DenseNet-169':[6, 12, 32, 32],
                   'DenseNet-201':[6, 12, 48, 32],
                   'DenseNet-265':[6, 12, 64, 48]}

#base_growth_rate = 32

model = KerasClassifier(build_fn = DenseNet, verbose=1)
search = GridSearchCV(model, hyperparameters, cv=4)

In [12]:
best_model = search.fit(x_total, y_total, epochs=30)

Epoch 1/30
4500/4500 [==============================] - 64s 14ms/step - loss: 1.9834 - acc: 0.2791
Epoch 2/30
4500/4500 [==============================] - 39s 9ms/step - loss: 1.3223 - acc: 0.5440
Epoch 3/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.9821 - acc: 0.6591
Epoch 4/30
4500/4500 [==============================] - 39s 9ms/step - loss: 0.8025 - acc: 0.7233
Epoch 5/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.6252 - acc: 0.7913
Epoch 6/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.5396 - acc: 0.8191
Epoch 7/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.4403 - acc: 0.8498
Epoch 8/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.3523 - acc: 0.8751
Epoch 9/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.3224 - acc: 0.8904
Epoch 10/30
4500/4500 [==============================] - 40s 9ms/step - loss: 0.2205 - acc: 0.9249
Epoch 11/30
4500/4

4500/4500 [==============================] - 41s 9ms/step - loss: 0.0514 - acc: 0.9824
Epoch 23/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0680 - acc: 0.9778
Epoch 24/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0576 - acc: 0.9798
Epoch 25/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0755 - acc: 0.9738
Epoch 26/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0615 - acc: 0.9787
Epoch 27/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0467 - acc: 0.9838
Epoch 28/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.0704 - acc: 0.9760
Epoch 29/30
4500/4500 [==============================] - 41s 9ms/step - loss: 0.1161 - acc: 0.9593
Epoch 30/30
1500/1500 [==============================] - 9s 6ms/step
Epoch 1/30
4500/4500 [==============================] - 76s 17ms/step - loss: 1.9169 - acc: 0.2960
Epoch 2/30
4500/4500 [==============================

4500/4500 [==============================] - 22s 5ms/step - loss: 0.1602 - acc: 0.9444
Epoch 14/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.1022 - acc: 0.9662
Epoch 15/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.1271 - acc: 0.9556
Epoch 16/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.1042 - acc: 0.9667
Epoch 17/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0983 - acc: 0.9651
Epoch 18/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.1206 - acc: 0.9596
Epoch 19/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.1571 - acc: 0.9500
Epoch 20/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0817 - acc: 0.9711
Epoch 21/30
4500/4500 [==============================] - 21s 5ms/step - loss: 0.0557 - acc: 0.9798
Epoch 22/30
4500/4500 [==============================] - 22s 5ms/step - loss: 0.0666 - acc: 0.9784
Epoch 23/30
4500/4500 

4500/4500 [==============================] - 44s 10ms/step - loss: 0.6733 - acc: 0.7742
Epoch 5/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.5631 - acc: 0.8142
Epoch 6/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.4678 - acc: 0.8422
Epoch 7/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.3486 - acc: 0.8796
Epoch 8/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.2869 - acc: 0.8958
Epoch 9/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.2676 - acc: 0.9018
Epoch 10/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.2231 - acc: 0.9253
Epoch 11/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.2455 - acc: 0.9169
Epoch 12/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.1735 - acc: 0.9373
Epoch 13/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.1277 - acc: 0.9589
Epoch 14/30
4500/

4500/4500 [==============================] - 45s 10ms/step - loss: 0.1015 - acc: 0.9684
Epoch 26/30
4500/4500 [==============================] - 45s 10ms/step - loss: 0.0845 - acc: 0.9702
Epoch 27/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.0548 - acc: 0.9836
Epoch 28/30
4500/4500 [==============================] - 45s 10ms/step - loss: 0.0437 - acc: 0.9869
Epoch 29/30
4500/4500 [==============================] - 44s 10ms/step - loss: 0.0793 - acc: 0.9747
Epoch 30/30
1500/1500 [==============================] - 43s 28ms/step
Epoch 1/30
4500/4500 [==============================] - 161s 36ms/step - loss: 1.8670 - acc: 0.3418
Epoch 2/30
4500/4500 [==============================] - 46s 10ms/step - loss: 1.1431 - acc: 0.5976
Epoch 3/30
4500/4500 [==============================] - 46s 10ms/step - loss: 0.8207 - acc: 0.7233
Epoch 4/30
4500/4500 [==============================] - 46s 10ms/step - loss: 0.6415 - acc: 0.7789
Epoch 5/30
4500/4500 [======================

4500/4500 [==============================] - 25s 6ms/step - loss: 0.1042 - acc: 0.9640
Epoch 16/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.1294 - acc: 0.9591
Epoch 17/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0994 - acc: 0.9656
Epoch 18/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0738 - acc: 0.9736
Epoch 19/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0641 - acc: 0.9782
Epoch 20/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0964 - acc: 0.9684
Epoch 21/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0711 - acc: 0.9776
Epoch 22/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0258 - acc: 0.9916
Epoch 23/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0227 - acc: 0.9922
Epoch 24/30
4500/4500 [==============================] - 25s 6ms/step - loss: 0.0498 - acc: 0.9829
Epoch 25/30
4500/4500 

4500/4500 [==============================] - 52s 11ms/step - loss: 0.5321 - acc: 0.8242
Epoch 7/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.4153 - acc: 0.8584
Epoch 8/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.3511 - acc: 0.8836
Epoch 9/30
4500/4500 [==============================] - 52s 11ms/step - loss: 0.3232 - acc: 0.8931
Epoch 10/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.2838 - acc: 0.9064
Epoch 11/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.2337 - acc: 0.9247
Epoch 12/30
4500/4500 [==============================] - 52s 11ms/step - loss: 0.2223 - acc: 0.9260
Epoch 13/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.1262 - acc: 0.9560
Epoch 14/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.2180 - acc: 0.9338
Epoch 15/30
4500/4500 [==============================] - 51s 11ms/step - loss: 0.1480 - acc: 0.9538
Epoch 16/30
450

4500/4500 [==============================] - 52s 12ms/step - loss: 0.1197 - acc: 0.9611
Epoch 28/30
4500/4500 [==============================] - 52s 12ms/step - loss: 0.1053 - acc: 0.9649
Epoch 29/30
4500/4500 [==============================] - 52s 12ms/step - loss: 0.0770 - acc: 0.9758
Epoch 30/30
1500/1500 [==============================] - 125s 83ms/step
Epoch 1/30
4500/4500 [==============================] - 354s 79ms/step - loss: 1.9237 - acc: 0.3451
Epoch 2/30
4500/4500 [==============================] - 54s 12ms/step - loss: 1.1505 - acc: 0.6082
Epoch 3/30
4500/4500 [==============================] - 54s 12ms/step - loss: 0.8518 - acc: 0.7067
Epoch 4/30
4500/4500 [==============================] - 54s 12ms/step - loss: 0.7144 - acc: 0.7653
Epoch 5/30
4500/4500 [==============================] - 54s 12ms/step - loss: 0.5448 - acc: 0.8111
Epoch 6/30
4500/4500 [==============================] - 54s 12ms/step - loss: 0.4799 - acc: 0.8364
Epoch 7/30
4500/4500 [=======================

4500/4500 [==============================] - 30s 7ms/step - loss: 0.0750 - acc: 0.9731
Epoch 18/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.1355 - acc: 0.9549
Epoch 19/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.1323 - acc: 0.9600
Epoch 20/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.1399 - acc: 0.9529
Epoch 21/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0743 - acc: 0.9747
Epoch 22/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0381 - acc: 0.9864
Epoch 23/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0658 - acc: 0.9791
Epoch 24/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0630 - acc: 0.9802
Epoch 25/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0898 - acc: 0.9676
Epoch 26/30
4500/4500 [==============================] - 30s 7ms/step - loss: 0.0865 - acc: 0.9698
Epoch 27/30
4500/4500 

6000/6000 [==============================] - 43s 7ms/step - loss: 0.2661 - acc: 0.9090
Epoch 9/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1605 - acc: 0.9438
Epoch 10/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1667 - acc: 0.9400
Epoch 11/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1782 - acc: 0.9363
Epoch 12/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1222 - acc: 0.9595
Epoch 13/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1190 - acc: 0.9578
Epoch 14/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1113 - acc: 0.9643
Epoch 15/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.1193 - acc: 0.9587
Epoch 16/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.0614 - acc: 0.9797
Epoch 17/30
6000/6000 [==============================] - 43s 7ms/step - loss: 0.0901 - acc: 0.9710
Epoch 18/30
6000/6000 [

In [15]:
print(best_model.best_params_)

{'base_growth_rate': 32, 'batch_size': 64}


In [14]:
pred = best_model.predict(x_test)

NameError: name 'x_test' is not defined